In [1]:
import ijson
import itertools
import random
import numpy as np
import sys, os
import pandas as pd 
import torch
from torchsummary import summary
from torchtext import data
import torch.nn as nn
import torch.utils.data
from torch.utils.data import Dataset, TensorDataset,DataLoader, RandomSampler
from torch.utils.tensorboard import SummaryWriter
import torchvision
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
from tqdm import tqdm, tqdm_notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings(action='once')
import pickle
from apex import amp
import shutil
import time

%load_ext autoreload
%autoreload 2
%matplotlib inline

SEED = 7219
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

ModuleNotFoundError: No module named 'past'

In [ ]:
# Import transformers specific packages
from transformers import BertTokenizer, BertModel, BertConfig
from transformers import  BertForSequenceClassification, BertForTokenClassification
from transformers import AdamW,get_linear_schedule_with_warmup

In [ ]:
# Set the device and empty cache 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.empty_cache()
device

- Export trained model for inference on TX2
- Add tensorboard summaries for monitoring training 
- Add accuracy, F1 score, log loss metric calculations
- Add architectural layers (freeze BERT layers, add new layers)
- Create pipeline


In [ ]:
# Class for model training and inference
class Bert_Model():
    def __init__(self,train_df,test_df,bert_model_name,bert_model_path,
                tokenizer,
                max_seq_length=128,seed=1234):
        
        if max_seq_length > tokenizer.max_model_input_sizes[bert_model_name]:
            print("Max sequence length specified > 512!!... resetting to 128")
            print("If you don't want this then set max_seq_length to <= 512")
            self._MAX_SEQUENCE_LENGTH = 128
        else:
            self._MAX_SEQUENCE_LENGTH = max_seq_length
        self._SEED = seed
        self._WORK_DIR = "/root/models/Tranformer_based/workingdir/"
        self._bert_model_path=bert_model_path
        self._bert_model_name=bert_model_name
        self._train_data=train_df
        self._test_size=test_df
        self._tokenizer = tokenizer
        self._training_stats = []

    def tokenize(self,text_array):
        ''' Returns tokenized IDs and attention mask
        The transformers encode_plus method returns the following:
        {
        input_ids: list[int],
        token_type_ids: list[int] if return_token_type_ids is True (default)
        attention_mask: list[int] if return_attention_mask is True (default)
        overflowing_tokens: list[int] if a ``max_length`` is specified and return_overflowing_tokens is True
        num_truncated_tokens: int if a ``max_length`` is specified and return_overflowing_tokens is True
        special_tokens_mask: list[int] if ``add_special_tokens`` if set to ``True`` and return_special_tokens_mask is True
        }'''
        all_tokens=[]
        all_attention_mask=[]
        for i,text in enumerate(tqdm_notebook(text_array)):
            encoded = tokenizer.encode_plus(
                           text, 
                           add_special_tokens=True,
                           max_length=self._MAX_SEQUENCE_LENGTH,
                           pad_to_max_length=True)
            tokens = torch.tensor(encoded['input_ids'])
            attention_mask = torch.tensor(encoded['attention_mask'])
            all_tokens.append(tokens)
            all_attention_mask.append(attention_mask)
        return all_tokens,all_attention_mask
     
    def initialize_model_for_training(self,dataset_len,EPOCHS=1,model_seed=21000,lr=2e-5,batch_size=32,
                                      accumulation_steps=2):
        # Setup model parameters
        np.random.seed(model_seed)
        torch.manual_seed(model_seed)
        torch.cuda.manual_seed(model_seed)
        torch.backends.cudnn.deterministic = True

        # Empty cache
        torch.cuda.empty_cache()
        model = BertForSequenceClassification.from_pretrained(self._bert_model_path,
                                                              cache_dir=None,
                                                              num_labels=2,
                                                              output_attentions = False, 
                                                              output_hidden_states = False)
        model.zero_grad()
        model = model.to(device)
        param_optimizer = list(model.named_parameters())
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
            ]
        num_train_optimization_steps = int(EPOCHS*dataset_len/batch_size/accumulation_steps)
        optimizer = AdamW(optimizer_grouped_parameters, lr=lr,eps=1e-8,correct_bias=False)  # To reproduce BertAdam specific behavior set correct_bias=False
        scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps=0,num_training_steps=num_train_optimization_steps)  # PyTorch scheduler
        model, optimizer = amp.initialize(model,optimizer,opt_level="O1",verbosity=0)
        model=model.train()
        return model,optimizer,scheduler,EPOCHS
    
    def run_training(self,model,train_dataLoader,valid_dataLoader,optimizer,scheduler,EPOCHS=1,batch_size=32,accumulation_steps=2):
        tq = tqdm_notebook(range(EPOCHS))
        training_stats=[]
      
        for epoch in tq:
            avg_loss = 0.
            avg_accuracy = 0.
            lossf=None
            tk0 = tqdm_notebook(enumerate(train_dataLoader),total=len(train_dataLoader),leave=False)
            optimizer.zero_grad()   
            for step,(x_batch, attn_mask,y_batch) in tk0:
                outputs = model(x_batch.to(device), 
                                token_type_ids=None, 
                                attention_mask=attn_mask.to(device), 
                                labels=y_batch.to(device))
                loss, y_pred = outputs
#                 loss =  F.binary_cross_entropy_with_logits(y_pred,y_batch.to(device))
#                 with amp.scale_loss(loss, optimizer) as scaled_loss:
#                     scaled_loss.backward()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # clip the norm to 1.0 to prevent exploding gradients
                if (step+1) % accumulation_steps == 0:          # Wait for several backward steps
                    model.zero_grad()
                    optimizer.step()                            # Now we can do an optimizer step
                    scheduler.step()
                    optimizer.zero_grad()
                if lossf:
                    lossf = 0.98*lossf+0.02*loss.item()
                else:
                    lossf = loss.item()
                tk0.set_postfix(loss = lossf)
                avg_loss += lossf / len(train_dataLoader)
                avg_accuracy += torch.mean(((torch.sigmoid(y_pred[:,0])>0.5) == (y_batch>0.5).to(device)).to(torch.float) ).item()/len(train_dataLoader)
                train_loss.append(avg_loss)
                train_accuracy.append(avg_accuracy)    
                training_stats
            tq.set_postfix(avg_loss=avg_loss,avg_accuracy=avg_accuracy)
            # Run evaluation
            eval_loss,eval_accuracy=self.run_eval(model,valid_dataLoader,epoch)
            print("Epoch Number:%d||Train Loss:%0.5f||Train Accuracy:%0.5f" %(epoch+1,avg_loss,avg_accuracy))
            print("Epoch Number:%d||Valid Loss:%0.5f||Valid Accuracy:%0.5f" %(epoch+1,eval_loss,eval_accuracy))
            valid_loss.append(eval_loss)
            valid_accuracy.append(eval_accuracy)
        return model
    
    def run_eval(self,model,valid_dataLoader,epoch):
        tq = tqdm_notebook(range(1))
        training_star
        avg_loss = 0.
        eval_accuracy = 0.
        eval_loss = 0.
        nb_eval_steps = 0
        lossf=None
        tk0 = tqdm_notebook(enumerate(valid_dataLoader),total=len(valid_dataLoader),leave=False)
        for step,(x_batch, attn_mask,y_batch) in tk0:
            model.eval()
            with torch.no_grad():
                outputs = model(x_batch.to(device), 
                                token_type_ids=None, 
                                attention_mask=attn_mask.to(device), 
                                labels=y_batch.to(device))
            loss, y_pred = outputs
            y_pred = y_pred.detach().cpu().numpy()
            label_ids = y_batch.to('cpu').numpy()
            tmp_eval_accuracy = self.flat_accuracy(y_pred, label_ids)
            if lossf:
                lossf = 0.98*lossf+0.02*loss.item()
            else:
                lossf = loss.item()
            # Accumulate the total accuracy.
            eval_loss += lossf/len(valid_dataLoader)
            eval_accuracy += tmp_eval_accuracy/len(valid_dataLoader)
            nb_eval_steps += 1
        avg_loss = eval_loss/nb_eval_steps
        avg_accuracy = eval_accuracy/nb_eval_steps
        tq.set_postfix(avg_accuracy=avg_accuracy)
        return avg_loss,avg_accuracy

     # Function to calculate the accuracy of predictions vs labels
    def flat_accuracy(self,preds, labels):
        pred_flat = np.argmax(preds, axis=1).flatten()
        labels_flat = labels.flatten()
        return np.sum(pred_flat == labels_flat) / len(labels_flat)
    
    def summaryWriter(self,model,):
        # Writer will output to ./runs/ directory by default
        writer = SummaryWriter()
        writer.add_graph(model)
        writer.add_scalar('Loss/train', np.random.random(), n_iter)
        
        
        
        writer.close()

In [5]:
class CreateDataset(Dataset):
    def __init__(self,data,atten_mask,labels):
        self._dataset = [[data[i],atten_mask[i],labels.values[i]] for i in range(0,len(data))]
    
    def __len__(self):
        return len(self._dataset)

    def __getitem__(self,idx):
        return self._dataset[idx]


In [6]:
# Define constants and paths

seed = 7843
bert_model_name = "bert-base-uncased"
# Convert TF checkpoint to pytorch chckpoint and then use as input to class object
bert_model_path = "/root/data/BERT/uncased_L-12_H-768_A-12-pytorch/"
# data_path = "/Users/suhasgupta/w251/mids-w251-final-project/data/nlp.cs.princeton.edu/SARC/2.0/pol/"
data_path = "/root/data/nlp.cs.princeton.edu/SARC/2.0/files/"
work_dir = "/root/models/Tranformer_based/workingdir/"

tokenizer = BertTokenizer.from_pretrained(bert_model_name, cache_dir=None,do_lower_case=True)
max_seq_len = 128

# Load and check the dataset from files on disk 
# train_file_name = data_path+'balanced_train.csv'
# test_file_name  = data_path+'balanced_test.csv'

train_file_name = data_path+'small_train.csv'
test_file_name  = data_path+'small_test.csv'

all_train_df = pd.read_csv(train_file_name)
test_df = pd.read_csv(test_file_name)

# Create a train, valid split
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(all_train_df, test_size=0.2,random_state=seed)

train_data   = train_df.text.fillna("DUMMY_VALUE")
train_labels = train_df.label
valid_data  = valid_df.text.fillna("DUMMY_VALUE")
valid_labels = valid_df.label
test_data  = test_df.text.fillna("DUMMY_VALUE")
test_labels = test_df.label

train_size,valid_size,test_size = len(train_df),len(valid_df),len(test_df)
print(train_size,valid_size,test_size)

799 200 999


/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [7]:
# Create a model object 
bert_model1=Bert_Model(train_df=train_df,
                      test_df=test_df,
                      bert_model_name=bert_model_name,
                      bert_model_path=bert_model_path,
                      tokenizer=tokenizer,
                      max_seq_length=max_seq_len)

In [8]:
%timeit
train_data_tokenized,train_attention_mask = bert_model1.tokenize(train_data)
valid_data_tokenized,valid_attention_mask = bert_model1.tokenize(valid_data)
test_data_tokenized,test_attention_mask = bert_model1.tokenize(test_data)


In [9]:
type(train_data_tokenized)

list

In [10]:
train_dataset = CreateDataset(train_data_tokenized,train_attention_mask,train_labels)
train_sampler = RandomSampler(train_dataset)
valid_dataset = CreateDataset(valid_data_tokenized,valid_attention_mask,valid_labels)
valid_sampler = RandomSampler(valid_dataset)

In [11]:
# train_dataset[0]
# valid_dataset[10]

In [15]:
# Parameters for cloud GPU
'''
train_params = {'batch_size': 128,
          'shuffle': False,
          'num_workers': 16}
valid_params = {'batch_size': 128,
          'shuffle': False,
          'num_workers': 16}
'''
# Parameters for local GPU
train_params = {'batch_size': 32,
          'shuffle': False,
          'num_workers': 16}
valid_params = {'batch_size': 8,
          'shuffle': False,
          'num_workers': 16}

max_epochs = 4

# print("Expected number of batches:", int(len(train_data_tokenized)/params['batch_size']))
train_dataLoader = torch.utils.data.DataLoader(train_dataset,sampler=train_sampler,batch_size=train_params['batch_size'],
                                         shuffle=train_params['shuffle'],
                                         num_workers=train_params['num_workers'],
                                         pin_memory=False,drop_last=True)
valid_dataLoader = torch.utils.data.DataLoader(valid_dataset,sampler=valid_sampler,batch_size=valid_params['batch_size'],
                                         shuffle=valid_params['shuffle'],
                                         num_workers=valid_params['num_workers'],
                                         pin_memory=False,drop_last=True)
print("Generated number of training batches:%d" %len(train_dataLoader))
print("Generated number of validation batches:%d" %len(valid_dataLoader))


"\ntrain_params = {'batch_size': 128,\n          'shuffle': False,\n          'num_workers': 16}\nvalid_params = {'batch_size': 128,\n          'shuffle': False,\n          'num_workers': 16}\n"

Generated number of training batches:99
Generated number of validation batches:25


#### Model Initialization and Training 

In [16]:
# Initialize the model
model,optimizer,scheduler,EPOCHS = bert_model1.initialize_model_for_training(len(train_dataLoader),
                                                                             EPOCHS=max_epochs)


- Add mpi run capability for distributed processing


In [17]:
# Train 

start = time.time()
trained_model=bert_model1.run_training(model,train_dataLoader,valid_dataLoader,
                         optimizer=optimizer,scheduler=scheduler,EPOCHS=EPOCHS)

print("Training Time:", time.time()-start)

Epoch Number:1||Train Loss:0.75725||Train Accuracy:0.50253
Epoch Number:1||Valid Loss:0.02959||Valid Accuracy:0.02000


Epoch Number:2||Train Loss:0.77063||Train Accuracy:0.50253
Epoch Number:2||Valid Loss:0.02972||Valid Accuracy:0.02000


Epoch Number:3||Train Loss:0.76860||Train Accuracy:0.50126
Epoch Number:3||Valid Loss:0.02786||Valid Accuracy:0.02000


Epoch Number:4||Train Loss:0.76205||Train Accuracy:0.50126
Epoch Number:4||Valid Loss:0.03189||Valid Accuracy:0.02000
Training Time: 88.50984740257263


In [ ]:
vars(trained_model)

In [ ]:
trained_model.config